In [178]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
path='/home/student303/oyx/almama/'
data = pd.read_hdf(path+ 'somefeature/base_feature_all_0.h5')
print('Read Data Finish')

Read Data Finish


In [188]:
tmpmerge = data[['user_id','day','instance_id']].groupby(['user_id','day']).count().reset_index().rename(columns={'instance_id':'user_day_clik_cnt'})
data = pd.merge(data,tmpmerge, on=['user_id','day'],how='left')
del tmpmerge
gc.collect()

164

In [138]:
# devide mean of clik
def correct_data1(x,data,vision='1'):
    correct_mean=data[[x,'day']].groupby('day').mean().reset_index().rename(columns={x:'temp'})
    data=pd.merge(data,correct_mean,on='day',how='left')
    data[x + '_' +vision]=data[x]/data['temp']
    del data['temp']
    return data

In [139]:
# devide sum of clik
def correct_data2(x,data,vision='2'):
    correct_mean=data[[x,'day']].groupby('day').sum().reset_index().rename(columns={x:'temp'})
    data=pd.merge(data,correct_mean,on='day',how='left')
    data[x + '_' +vision]=data[x]/data['temp']
    del data['temp']
    return data

In [136]:
# min-max normalization
def correct_data3(x,data,vision='3'):
    
    merger = data[[x,'day']].groupby('day').max().reset_index().rename(columns={x:'tmp_max'})
    data = pd.merge(data,merger,on=['day'],how='left')
    merger = data[[x,'day']].groupby('day').min().reset_index().rename(columns={x:'tmp_min'})
    data = pd.merge(data,merger,on=['day'],how='left')
    data[x + '_' +vision] = (data[x] - data['tmp_min'])/(data['tmp_max'] - data['tmp_min']) 
    del data['tmp_max']
    del data['tmp_min']
    del merger
    gc.collect()
    return data

In [137]:
# Z-score normalization
def correct_data4(x,data,vision='4'):
    
    merger = data[[x,'day']].groupby('day').mean().reset_index().rename(columns={x:'tmp_mean'})
    data = pd.merge(data,merger,on=['day'],how='left')
    merger = data[[x,'day']].groupby('day').std().reset_index().rename(columns={x:'tmp_std'})
    data = pd.merge(data,merger,on=['day'],how='left')
    data[x + '_' +vision] = (data[x] - data['tmp_mean'])/data['tmp_std']
    del data['tmp_mean']
    del data['tmp_std']
    del merger
    gc.collect()
    return data

In [196]:
data=correct_data1('user_day_clik_cnt',data)
data=correct_data2('user_day_clik_cnt',data)
data=correct_data3('user_day_clik_cnt',data)
data=correct_data4('user_day_clik_cnt',data)

In [202]:
#Define featurelist
droplist=['instance_id',
          'item_category_list',
          'item_property_list',
          'user_id', 
          'context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade',
          'day', 'min', 'second',
         
          #'user_day_clik_cnt',
          'user_day_clik_cnt_1',
          'user_day_clik_cnt_2',
          'user_day_clik_cnt_3',
          'user_day_clik_cnt_4',
          
         # 'user_click_same_item',
          'user_click_same_item_1',
          'user_click_same_item_2',
          'user_click_same_item_3',
          'user_click_same_item_4',
          
          'user_day_diff_category',
          'user_day_diff_category_1',
         # 'user_day_diff_category_2',
          'user_day_diff_category_3',
          'user_day_diff_category_4'
          
         ]
featurelist=[i for i in data.columns if i not in droplist]


print('LGBM Training, Feature dim : ',len(featurelist))

params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':4,
    #'num_leaves': 6,
    #'min_data_in_leaf':1,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 1.0,
    'verbose': 0,
}

d_train=lgb.Dataset(data.loc[data.day < 24][featurelist], label=data.loc[data.day < 24]['is_trade'])
d_valid=lgb.Dataset(data.loc[data.day == 24][featurelist], label=data.loc[data.day == 24]['is_trade'])
model = lgb.train(params, train_set=d_train, num_boost_round=600, valid_sets=[d_train,d_valid], \
              early_stopping_rounds=50, verbose_eval=30)
print(end='\n')
for i in featurelist:
    print(i)

LGBM Training, Feature dim :  23
Train until valid scores didn't improve in 50 rounds.
[30]	training's binary_logloss: 0.0997635	valid_1's binary_logloss: 0.0932022
[60]	training's binary_logloss: 0.0889659	valid_1's binary_logloss: 0.0817256
[90]	training's binary_logloss: 0.0875513	valid_1's binary_logloss: 0.0809783
[120]	training's binary_logloss: 0.0867432	valid_1's binary_logloss: 0.0808504
[150]	training's binary_logloss: 0.086097	valid_1's binary_logloss: 0.080806
[180]	training's binary_logloss: 0.0855213	valid_1's binary_logloss: 0.0807606
[210]	training's binary_logloss: 0.0850481	valid_1's binary_logloss: 0.0807874
Early stopping, best iteration is:
[189]	training's binary_logloss: 0.0853498	valid_1's binary_logloss: 0.0807543

item_id
item_brand_id
item_city_id
item_price_level
item_sales_level
item_collected_level
item_pv_level
user_gender_id
user_age_level
user_occupation_id
user_star_level
context_page_id
shop_id
shop_review_num_level
shop_review_positive_rate
shop_star

In [ ]:
0.0807543
1 - 0.080704
2 - 0.080642
3 - 0.080953
4 - 0.0806829

In [116]:
#base 0.0816614
#add user_day_clik_cnt_1 0.0817182 除了每日的均值
#add user_day_clik_cnt_2 0.0816129 除了每日的点击数总和
#add user_day_clik_cnt_3 0.0816873 对每日做了min-max标准化
#add user_day_clik_cnt_4 0.0815966 对每日做了Z-score标准化
#add user_day_clik_cnt and user_day_clik_cnt_4  0.0815409 都加进去了了又降了？？

## 3-23公粮

In [130]:
data_3_23 = pd.read_hdf(path+ 'somefeature/base_feature_data_del_add_userclicksameitem.h5')
data = data_3_23

In [143]:
data = correct_data4('user_click_same_item',data)
data = correct_data3('user_click_same_item',data)
data = correct_data2('user_click_same_item',data)
data = correct_data1('user_click_same_item',data)

In [153]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'item_category_hash', 'day',
       'hour', 'min', 'second', 'user_lastclicktime_interval',
       'user_click_same_item', 'user_click_same_item_4',
       'user_click_same_item_3', 'user_click_same_item_2',
       'user_click_same_item_1'],
      dtype='object')

In [ ]:
#base 0.0805906
#add user_click_same_item_1 0.0806825
#add user_click_same_item_2 0.0805639
#add user_click_same_item_3 0.080595
#add user_click_same_item_4 0.0805349

## emmmmm

In [182]:
data_3_23_Ture = pd.read_hdf(path+ 'somefeature/0322base_feature_data_del_add_userclicksameitem.h5')

In [183]:
data = data_3_23_Ture

In [165]:
data = correct_data4('user_day_diff_category',data)
data = correct_data3('user_day_diff_category',data)
data = correct_data2('user_day_diff_category',data)
data = correct_data1('user_day_diff_category',data)

In [177]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'item_category_hash', 'day',
       'hour', 'min', 'second', 'user_lastclicktime_interval',
       'user_click_same_item', 'user_day_diff_category_2'],
      dtype='object')

In [ ]:
#base 0.0806006
#divide mean  0.0801435
#divide sum  0.0800458
#min-max     0.0801064
#Z-score     0.0801928

In [176]:
data.drop(['user_day_diff_category','user_day_diff_category_1','user_day_diff_category_3','user_day_diff_category_4'],axis=1,inplace=True)
data.to_hdf(path + 'somefeature/0323base_feature_data_del_usercliksameitem_correct2.h5',key='a')

In [185]:
data.drop(['user_click_same_item','user_day_diff_category'],axis=1,inplace=True)

In [205]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'item_category_hash', 'day',
       'hour', 'min', 'second', 'user_lastclicktime_interval',
       'user_day_clik_cnt_4'],
      dtype='object')

In [204]:
data.drop(['user_day_clik_cnt','user_day_clik_cnt_1','user_day_clik_cnt_2','user_day_clik_cnt_3',],axis=1,inplace=True)

In [206]:
data.to_hdf(path + 'somefeature/0323base_feature_data_del_dailycliktimes_correct4.h5',key='a')